# MASH analysis of GTEx data, Urbut 2017

This notebook performs the "master" analysis for Urbut et al 2017, using [`mashr-paper`](https://github.com/stephenslab/mashr-paper) version of MASH implementation.

**FIXME: add more intro here**:

1. Explain the input / output -- make it clear that we've omitted the steps tha converts summary stats from gtex portal to mashable format, and that we did not apply the inference to all the gene-snp pairs.
2. Explain that we have refactored to code to `mashr` repo. We may or may not want to point to [the analysis that uses the new `mashr` code](http://stephenslab.github.io/gtex-eqtls/analysis/20171002_MASH_V8.html).

## Run this notebook
For repeated runs it might be easier to execute from commandline instead of in Jupyter:

```bash
sos run code/master_mash_analysis.ipynb # --data ... --cwd ...
```

The notebook runs default setting. Additionally I run it for dataset after LD pruning (for LD related discussion in supplemental information):

```bash
sos run code/master_mash_analysis.ipynb --data data/MatrixEQTLSumStats.Portable.ld2.Z.rds
```

Some notes about the code used for the core analysis:
    
    1. `ms=deconvolution.em.with.bovy(t.stat,factor.mat,v.j,lambda.mat,K=3,P=3)`

produces an object with the denoised matrices for feeding into the
*mash* covariance code. The *factor.mat* and *lambda.mat* called
within have been produced by SFA and are single rank factors and
loadings approximating the empirical covariance.

    2. `covmat=compute.hm.covmat.all.max.step(b.hat=z.stat,se.hat=v.j,t.stat=z.stat,Q=5,lambda.mat,A=A,factor.mat,max.step=max.step,zero=TRUE)$covmat` 

produces a list of covariance matrices entitled *covmat"A".rds* upon
which to base the mixture of multivariate normals.

    3. `compute.hm.train.log.lik(train.b = train.z,se.train = train.v,covmat = covmat,A,pen=TRUE)`

computes the HM weights on training datauses the set of randomly chosen genes to train our model and produces
a matrix of likelihoods and corresponding hierarchical weights, as well as the mixture proportions.

    4. `weightedquants=lapply(seq(1:nrow(z.stat)),function(j){total.quant.per.snp(j,covmat,b.gp.hat=z.stat,se.gp.hat = v.j,pis,A,checkpoint = FALSE)})`

produces files containing the posterior means, upper and lower
tail probabilities, null probabilites, and lfsr for all J genes across
44 conditions.

In [1]:
[global]
parameter: cwd = path('./gtex6_workflow_output')
parameter: data = path("data/MatrixEQTLSumStats.Portable.Z.rds")
# path configured to use with root privilege, for `docker` 
mashr_src = path("/opt/mash-paper/main.R")
sfa_exe = "/opt/sfa/bin/sfa_linux"

## Covariance pattern discovery
This obtains covariance matrices, ie, the priors, for `mash` model.

### SFA
We analyze data with SFA. The cell below downloads SFA software and run it on data with rank `K = 5`.

In [2]:
%sosrun sfa
[sfa_download: provides = sfa_exe]
download: decompress = True, dest_dir = cwd
    http://stephenslab.uchicago.edu/assets/software/sfa/sfa1.0.tar.gz

[sfa]
# Perform SFA analysis (time estimate: ~3min)
depends: sfa_exe
K = 5
tmpfile = path(f"{cwd:a}/{data:bn}.max.txt")
input: f"{data:a}"
output: f"{cwd:a}/{_input:bn}.sfa.rds"
R: expand = "${ }", stdout = f"{_output}.log", workdir = cwd
    z = readRDS(${_input:r})$test.z
    write.table(z, ${tmpfile:r}, col.names=F,row.names=F)
    cmd = paste0('${sfa_exe} -gen ${tmpfile} -g ', dim(z)[1], ' -n ', dim(z)[2], 
                 ' -k ${K} -iter 50 -rand 999 -o ${_output:bn}')
    system(cmd)
    saveRDS(list(F = read.table("${_output:n}_F.out"),
                lambda = read.table("${_output:n}_lambda.out"),
                sigma2 = read.table("${_output:n}_sigma2.out"),
                alpha = read.table("${_output:n}_alpha.out")), ${_output:r})
bash: workdir = cwd
    rm -f *{_F.out,_lambda.out,_sigma2.out,_alpha.out}

###  Create and refine multi-rank covariance matrices
Here we create 3 covariance matrices:

* SFA (rank 5, previously computed)
* SVD (rank 3, to be computed)
* Empirical covariance

and apply [Extreme Deconvolution](https://github.com/jobovy/extreme-deconvolution) to refine the matrices. We observed that Extreme Deconvolution perserves rank.

In [3]:
[mashr_download: provides = mashr_src]
task: workdir = cwd
download: decompress = True
    https://github.com/stephenslab/mashr-paper/archive/v0.2-1.zip

[mash_1: shared = {'mash_input': '_input'}]
# Compute data-driven prior matrices (time estimate: ~1.5hr)
depends: R_library("ExtremeDeconvolution"), mashr_src
K = 3
P = 3
input: f"{data:a}", f"{cwd:a}/{data:bn}.sfa.rds"
output: f"{cwd:a}/{data:bn}.coved.K{K}.P{P}.rds"
R: expand = "${ }", workdir = cwd
    setwd(${mashr_src:dar})
    ret = sapply(list.files(pattern = "*.R"), source, .GlobalEnv)
    setwd(${cwd:ar})
    dat = readRDS(${_input[0]:r})
    t.stat = dat$test.z
    mean.mat = matrix(rep(0,ncol(t.stat)*nrow(t.stat)),ncol=ncol(t.stat),nrow=nrow(t.stat))
    s.j = matrix(rep(1,ncol(t.stat)*nrow(t.stat)),ncol=ncol(t.stat),nrow=nrow(t.stat))
    v.mat = dat$vhat
    v.j=list()
    for(i in 1:nrow(t.stat)){v.j[[i]]=v.mat}
    K = ${K}
    P = ${P}
    R = ncol(t.stat)
    sfa = readRDS(${_input[1]:r})
    init.cov = init.covmat(t.stat=t.stat, factor.mat = as.matrix(sfa$F),lambda.mat = as.matrix(sfa$lambda), K=K,P=P)
    init.cov.list = list()
    for(i in 1:K){init.cov.list[[i]]=init.cov[i,,]}
    projection = list();for(l in 1:nrow(t.stat)){projection[[l]]=diag(1,R)}
    e = ExtremeDeconvolution::extreme_deconvolution(ydata=t.stat,ycovar=v.j,xamp=rep(1/K,K),xmean=mean.mat,xcovar=init.cov.list,fixmean=T,projection=projection)
    true.covs = array(dim=c(K,R,R))
    for(i in 1:K){true.covs[i,,]=e$xcovar[[i]]}
    saveRDS(list(true.covs=true.covs,pi=e$xamp), ${_output:r})

### Add in single-rank covariance matrices
Now additionally we include 2 other types of covariance matrices:
* canonical configurations (aka `bmalite`)
* single rank SFA

We also expand the list of matrices by grid. At the end of this step (cell below) we are ready to fit the mash model.

In [4]:
[mash_2: shared = {'prior_matrices': '_output'}]
# Add in canonical configurations and single rank SFA priors (time estimate: <1min)
depends: sos_variable('mash_input')
output: f"{_input:n}.lite.single.expanded.rds"
R: expand = "${ }", workdir = cwd
    setwd(${mashr_src:dar})
    ret = sapply(list.files(pattern = "*.R"), source, .GlobalEnv)
    setwd(${cwd:ar})
    dat = readRDS(${mash_input[0]:r})
    z.stat = dat$test.z
    rownames(z.stat) = NULL
    colnames(z.stat) = NULL
    v.mat = dat$vhat
    s.j = matrix(rep(1,ncol(z.stat)*nrow(z.stat)),ncol=ncol(z.stat),nrow=nrow(z.stat))
    sfa = readRDS(${mash_input[1]:r})
    res = compute.hm.covmat.all.max.step(b.hat=z.stat,se.hat=s.j,
                                          t.stat=z.stat,Q=5,
                                          lambda.mat=as.matrix(sfa$lambda),
                                          A='.remove_before_rerun',
                                          factor.mat=as.matrix(sfa$F),
                                          max.step=readRDS(${_input:r}),
                                          zero=TRUE)
    saveRDS(res, ${_output:r})

bash: workdir = cwd
    rm -f *.remove_before_rerun.rds

## Fit MASH mixture model
Using a training set, the cell below computes the weights for input covariance matrices (priors) in MASH mixture. The output contains matrix of log-likelihoods as well as weights learned from the hierarchical model.

In [5]:
[mash_3]
# Fit MASH mixture model (time estimate: )
depends: R_library("SQUAREM")
output: f"{_input:n}.pihat.rds", f"{_input:n}.loglik.rds"
R: expand = "${ }", workdir = cwd
    library("SQUAREM")
    setwd(${mashr_src:dar})
    ret = sapply(list.files(pattern = "*.R"), source, .GlobalEnv)
    setwd(${cwd:ar})
    dat = readRDS(${mash_input[0]:r})
    v.mat = dat$vhat
    covmat = readRDS(${_input:r})$covmat
    train.z = as.matrix(dat$train.z)
    rownames(train.z) = NULL
    colnames(train.z) = NULL
    train.v = train.z/train.z
    res = compute.hm.train.log.lik.pen.vmat(train.b=train.z,
                                            covmat=covmat,
                                            A='.remove_before_rerun', 
                                            pen=1,
                                            train.s=train.v,
                                            cormat=v.mat)
    saveRDS(res$pis, ${_output[0]:r})
    saveRDS(res$lik.mat, ${_output[1]:r})

bash:
    rm -f *.remove_before_rerun.rds

## Posterior inference
Applying hyperparameters learned from the training set to the test set, the cell below computes posterior quantities. 

In [6]:
[mash_4]
# Posterior inference on the "top" set of gene-snp pairs (time estimate: )
depends: sos_variable('prior_matrices')
output: f"{_input[0]:nn}.posterior.rds"
R: expand = "${ }", workdir = cwd
    setwd(${mashr_src:dar})
    ret = sapply(list.files(pattern = "*.R"), source, .GlobalEnv)
    setwd(${cwd:ar})
    dat = readRDS(${mash_input[0]:r})
    z.stat = dat$test.z
    v.mat = dat$vhat
    s.j = matrix(rep(1,ncol(z.stat)*nrow(z.stat)),ncol=ncol(z.stat),nrow=nrow(z.stat))
    pis = readRDS(${_input[0]:r})$pihat
    covmat = readRDS(${prior_matrices:r})$covmat
    res = lapply(seq(1:nrow(z.stat)), function(j){
        total.quant.per.snp.with.vmat(j=j, covmat=covmat, 
                                      b.gp.hat=z.stat, 
                                      cormat=v.mat, 
                                      se.gp.hat=s.j, 
                                      pis=pis, 
                                      A='remove_before_rerun', 
                                      checkpoint=TRUE)})
    # data formatting.
    out = do.call(Map, c(f = rbind, res))
    saveRDS(out, ${_output:r})

bash:
    rm -f *.remove_before_rerun.rds

Now MASH analysis is complete. I will use a separate notebook to summarize, plot and visualize the result of analysis.

## Export and run default pipeline

In [7]:
[export]
# Export notebook to HTML file
input: [item for item in sys.argv if item.endswith('.ipynb')], group_by = 1, pattern = '{fd}/{fn}.ipynb'
output: expand_pattern(f'{cwd:a}/{_fn[0]}.html')
bash: expand = True, stderr = False
  sos convert {_input} {_output}
  
[default]
sos_run('export+sfa+mash')

In [8]:
%sessioninfo

SoS Version,0.9.13.3
